In [6]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
import wandb

random.seed(27)
np.random.seed(27)
plt.style.use('ggplot')

# Section 1: Pilot study in subset, used to find feasible hyperparameters

### Dataset - A (1/10) subset of cifar

We will choose a subset of the cifar10, and make sure there is class balance

In [7]:
# Pick dataset
ds = "cifar10"

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

if ds == "cifar10":
    train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_ds = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    #Normalise
    X_train = train_ds.data.astype(np.float32) / 255.0 
    y_train = np.array(train_ds.targets, dtype=np.int64)

    X_test  = test_ds.data.astype(np.float32) / 255.0
    y_test  = np.array(test_ds.targets, dtype=np.int64)

    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test  = X_test.reshape(X_test.shape[0], -1)

    # Flatten
    X_train = X_train.reshape(-1, np.prod(X_train.shape[1:]))
    X_test = X_test.reshape(-1, np.prod(X_test.shape[1:]))

elif ds == "mnist":
    mnist_train = datasets.MNIST(root="./data", train=True, download=True, transform=None)
    mnist_test = datasets.MNIST(root="./data", train=False, download=True, transform=None)

    X_train = mnist_train.data.to(torch.float32).numpy()
    y_train = mnist_train.targets.to(torch.int64).numpy()

    X_test  = mnist_test.data.to(torch.float32).numpy()
    y_test  = mnist_test.targets.to(torch.int64).numpy()

    # Normalize the pixel values (important for neural nets)
    X_train /= 255.0
    X_test /= 255.0

    # Flatten
    X_train = X_train.reshape(-1, np.prod(X_train.shape[1:]))
    X_test = X_test.reshape(-1, np.prod(X_test.shape[1:]))



subset_ratio = 1/10      # take 10% of data
num_classes = 3

def balanced_subset(X, y, ratio, num_classes=2):
    X = np.array(X)
    y = np.array(y)

    indices = []
    for c in range(num_classes):
        class_idx = np.where(y == c)[0]
        n_samples = int(len(class_idx) * ratio)

        # sample without replacement
        chosen = np.random.choice(class_idx, n_samples, replace=False)
        indices.append(chosen)

    indices = np.concatenate(indices)
    np.random.shuffle(indices)

    return X[indices], y[indices]

# Apply to train + test
X_train, y_train = balanced_subset(X_train, y_train, subset_ratio, num_classes)
X_test,  y_test  = balanced_subset(X_test,  y_test,  subset_ratio, num_classes)


def one_hot_encode(y, num_classes=10):
    y_encoded = np.zeros((num_classes, y.size))
    y_encoded[y, np.arange(y.size)] = 1
    return y_encoded

Y_train = one_hot_encode(y_train, 10)  # (10, 60000)
Y_test  = one_hot_encode(y_test, 10)   # (10, 10000)

# --------------------------------------------------
# Quick check
# --------------------------------------------------
print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
print("X_test: ", X_test.shape,  "Y_test: ",  Y_test.shape)

X_train: (1500, 3072) Y_train: (10, 1500)
X_test:  (300, 3072) Y_test:  (10, 300)


### Pilot study - we search for a feasible range of hyperparameters using sweeps

In [8]:
from NN.denseLayer import *
from NN.loss import *
from NN.optimizer import *

> Setup WANDB tracker, where we will perform a simple grid search of a VERY broad search

In [9]:
import matplotlib.pyplot as plt
import os
import wandb
from getAPI import retrieveApi

os.environ['WANDB_API_KEY'] = retrieveApi()

> Do hyperparameter sweep, on a very small subset of cifar-10

The code is based on the tutorials given in the sweeps documentation:
- https://docs.wandb.ai/models/sweeps/define-sweep-configuration#python-script-or-notebook
- https://docs.wandb.ai/models/tutorials/sweeps

In [ ]:

sweep_configuration = {
    "method": "random",
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1.0},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 30},
        "num_hidden_layers": {"distribution": "int_uniform", "min": 1, "max": 5},
        "n_hidden_units": {"distribution": "int_uniform", "min": 32, "max": 128},
        "l2_coeff": {"distribution": "uniform", "min": 0.0, "max": 0.001},
        "optimizer": {"values": ["sgd"]},
        "activation": {"values": ["relu", "tanh"]},
        "loss_fn": {"values": ["cross", "mse"]},
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="numpy_nn")


def accuracy(pred, y_true):
    return (pred.argmax(axis=0) == y_true.argmax(axis=0)).mean()


def get_activation(name):
    return ReLU() if name == "relu" else Tanh()


def get_optimizer(name, lr):
    return SGD(learning_rate=lr) if name == "sgd" else Adam(learning_rate=lr)


def get_loss(name):
    return CrossEntropyLoss() if name == "cross" else MSELoss()


def build_model(input_dim, cfg):
    layers = []
    act = get_activation(cfg.activation)
    h = cfg.n_hidden_units
    prev = input_dim

    for _ in range(cfg.num_hidden_layers):
        layers.append(DenseLayer(prev, h, activation=act, initializer=HeInitializer()))
        prev = h

    layers.append(DenseLayer(prev, 10, activation=Softmax(), initializer=NormalInitializer(mean=0, std=0.01)))
    return layers

def train_run():
    wandb.init()
    cfg = wandb.config
    wandb.log(dict(cfg))

    # Build layers with L2 regularization set per layer
    layers = []
    act = get_activation(cfg.activation)
    prev = X_train.shape[1]
    h = cfg.n_hidden_units

    for _ in range(cfg.num_hidden_layers):
        layers.append(DenseLayer(prev, h, activation=act, initializer=HeInitializer(), l2_coeff=cfg.l2_coeff))
        prev = h

    layers.append(DenseLayer(prev, 10, activation=Softmax(), initializer=NormalInitializer(mean=0, std=0.01)))

    optimizer = get_optimizer(cfg.optimizer, cfg.learning_rate)
    loss_fn = get_loss(cfg.loss_fn)
    m = X_train.shape[0]

    for epoch in range(cfg.epochs):
        perm = np.random.permutation(m)
        Xs = X_train[perm].T
        Ys = Y_train[:, perm]

        for i in range(0, m, cfg.batch_size):
            Xb = Xs[:, i:i+cfg.batch_size]
            Yb = Ys[:, i:i+cfg.batch_size]

            A = Xb
            for layer in layers:
                A = layer.forward(A)

            loss = loss_fn.forward(A, Yb)
            dA = loss_fn.backward(A, Yb)

            for layer in reversed(layers):
                dA, dW, db = layer.backward(dA)
                optimizer.update(layer, dW, db)

        acc = accuracy(A, Yb)
        wandb.log({"loss": loss, "accuracy": acc})
        print(f"Epoch {epoch+1}/{cfg.epochs}  Loss={loss:.4f}  Acc={acc:.4f}")

    wandb.finish()


wandb.agent(sweep_id, function=train_run, count=10)

Create sweep with ID: cr5oclrk
Sweep URL: https://wandb.ai/xanderbaatz-danmarks-tekniske-universitet-dtu/numpy_nn/sweeps/cr5oclrk


wandb: Agent Starting Run: wxngzj1q with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.0009612309151634822
wandb: 	learning_rate: 0.9150623076765048
wandb: 	loss_fn: mse
wandb: 	n_hidden_units: 55
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: sgd


Epoch 1/30  Loss=0.3548  Acc=0.3214
Epoch 2/30  Loss=0.3368  Acc=0.2857
Epoch 3/30  Loss=0.3370  Acc=0.3571
Epoch 4/30  Loss=0.3305  Acc=0.6071
Epoch 5/30  Loss=0.3321  Acc=0.3571
Epoch 6/30  Loss=0.3317  Acc=0.3929
Epoch 7/30  Loss=0.3308  Acc=0.5000
Epoch 8/30  Loss=0.3272  Acc=0.4286
Epoch 9/30  Loss=0.3350  Acc=0.3929
Epoch 10/30  Loss=0.3247  Acc=0.4643
Epoch 11/30  Loss=0.3262  Acc=0.4286
Epoch 12/30  Loss=0.3228  Acc=0.4286
Epoch 13/30  Loss=0.3154  Acc=0.6071
Epoch 14/30  Loss=0.3164  Acc=0.5714
Epoch 15/30  Loss=0.3255  Acc=0.4286
Epoch 16/30  Loss=0.3111  Acc=0.6071
Epoch 17/30  Loss=0.3068  Acc=0.6429
Epoch 18/30  Loss=0.3240  Acc=0.4286
Epoch 19/30  Loss=0.3138  Acc=0.4643
Epoch 20/30  Loss=0.3088  Acc=0.5357
Epoch 21/30  Loss=0.3260  Acc=0.4286
Epoch 22/30  Loss=0.3251  Acc=0.4286
Epoch 23/30  Loss=0.3146  Acc=0.3929
Epoch 24/30  Loss=0.3071  Acc=0.4643
Epoch 25/30  Loss=0.3005  Acc=0.5357
Epoch 26/30  Loss=0.2846  Acc=0.6071
Epoch 27/30  Loss=0.2891  Acc=0.5714
Epoch 28/3

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 30/30  Loss=0.2931  Acc=0.5357


...

wandb: Agent Starting Run: 6xqkl2u1 with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 30
wandb: 	l2_coeff: 4.4390556586005326e-05
wandb: 	learning_rate: 0.0026995599295566663
wandb: 	loss_fn: mse
wandb: 	n_hidden_units: 99
wandb: 	num_hidden_layers: 1
wandb: 	optimizer: sgd


Epoch 1/30  Loss=0.4511  Acc=0.3214
Epoch 2/30  Loss=0.4504  Acc=0.3214
Epoch 3/30  Loss=0.4465  Acc=0.1786
Epoch 4/30  Loss=0.4452  Acc=0.3571
Epoch 5/30  Loss=0.4463  Acc=0.3571
Epoch 6/30  Loss=0.4438  Acc=0.3214
Epoch 7/30  Loss=0.4440  Acc=0.2500
Epoch 8/30  Loss=0.4416  Acc=0.3929
Epoch 9/30  Loss=0.4478  Acc=0.0357
Epoch 10/30  Loss=0.4418  Acc=0.3571
Epoch 11/30  Loss=0.4441  Acc=0.1786
Epoch 12/30  Loss=0.4416  Acc=0.3214
Epoch 13/30  Loss=0.4430  Acc=0.1429
Epoch 14/30  Loss=0.4392  Acc=0.2143
Epoch 15/30  Loss=0.4366  Acc=0.3571
Epoch 16/30  Loss=0.4375  Acc=0.3214
Epoch 17/30  Loss=0.4372  Acc=0.3929
Epoch 18/30  Loss=0.4327  Acc=0.4286
Epoch 19/30  Loss=0.4313  Acc=0.6071
Epoch 20/30  Loss=0.4293  Acc=0.4286
Epoch 21/30  Loss=0.4316  Acc=0.3571
Epoch 22/30  Loss=0.4360  Acc=0.2143
Epoch 23/30  Loss=0.4361  Acc=0.1786
Epoch 24/30  Loss=0.4294  Acc=0.3214
Epoch 25/30  Loss=0.4271  Acc=0.3571
Epoch 26/30  Loss=0.4292  Acc=0.3571
Epoch 27/30  Loss=0.4297  Acc=0.2143
Epoch 28/3

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 29/30  Loss=0.4271  Acc=0.2857
Epoch 30/30  Loss=0.4242  Acc=0.3929


accuracy,▄▄▃▅▅▄▄▅▁▅▃▄▂▃▅▄▅▆█▆▅▃▃▄▅▅▃▅▄▅
batch_size,▁
epochs,▁
l2_coeff,▁
learning_rate,▁
loss,██▇▆▇▆▆▆▇▆▆▆▆▅▄▄▄▃▃▂▃▄▄▂▂▂▂▂▂▁
n_hidden_units,▁
num_hidden_layers,▁
accuracy,0.39286
activation,relu
batch_size,64


# Section 2: Hyperparameter study using grid search, random search and bayesian optimization

# Section 3: Validation of model